<a href="https://colab.research.google.com/github/kimdonggyu2008/cnn_emotional_recognization/blob/main/%EB%AA%A9%EC%86%8C%EB%A6%AC_%EA%B0%90%EC%A0%95_%EB%B6%84%EC%84%9D%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install matplotlib librosa

In [ ]:
import pandas as pd
import numpy as np

import os
import sys

import librosa#웨이브파일 관련 라이브러리
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split#지표 관련 라이브러리

from IPython.display import Audio #라이브러리 시각화

from tensorflow.python.keras.utils import np_utils
import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint


import warnings
if not sys.warnoptions:
  warnings.simplefilter("ignore")
warnings.filterwarnings("ignore",category=DeprecationWarning)

ravness


예시 데이터의 구분방법

데이터 유형 - 말? 노래? - 어떤 감정? - 감정 세기 - 어떤 대사? - 반복 횟수 - 성우
1. Ravdess Dataframe
Here is the filename identifiers as per the official RAVDESS website:

Modality (01 = full-AV, 02 = video-only, 03 = audio-only).

Vocal channel (01 = speech, 02 = song).

Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).

Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.

Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").

Repetition (01 = 1st repetition, 02 = 2nd repetition).

Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

So, here's an example of an audio filename. 02-01-06-01-02-01-12.mp4 This means the meta data for the audio file is:

Video-only (02)
Speech (01)
Fearful (06)
Normal intensity (01)
Statement "dogs" (02)
1st Repetition (01)
12th Actor (12) - Female (as the actor ID number is even)


In [ ]:
Ravdess="/content/drive/MyDrive/archive/audio_speech_actors_01-24/"
Tess="/content/drive/MyDrive/TESS Toronto emotional speech set data/"

In [ ]:
ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []#감정 종류
file_path = []#파일 경로
for dir in ravdess_directory_list:
  actor = os.listdir(Ravdess + dir)
  for file in actor:#각 성우에 대한 데이터 가져옴
    part=file.split('.')[0]#각 파일이름 에서 데이터유형 가져옴
    part=part.split('-')#'-'로 나눔
    file_emotion.append(int(part[2]))# 감정 구분
    file_path.append(Ravdess+dir+'/'+file)

emotion_df=pd.DataFrame(file_emotion,columns=['Emotions'])
path_df=pd.DataFrame(file_path,columns=['Path'])
Ravdess_df=pd.concat([emotion_df,path_df],axis=1)

Ravdess_df.Emotions.replace({1:'neutral',2:'calm',3:'happy',4:'sad',5:'angry',6:'fear',7:'disgust',8:'surprise'},inplace=True)
#숫자로 표현된 감정을 글자로 변환

tess dataset


In [ ]:
tess_directory_list = os.listdir(Tess)

file_emotion = []
file_path = []

for dir in tess_directory_list:
    directories = os.listdir(Tess + dir)
    for file in directories:
        part = file.split('.')[0]
        part = part.split('_')[2]
        if part=='ps':
            file_emotion.append('surprise')
        else:
            file_emotion.append(part)
        file_path.append(Tess + dir + '/' + file)

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Tess_df = pd.concat([emotion_df, path_df], axis=1)

In [ ]:
data_path=pd.concat([Ravdess_df,Tess_df],axis=0)#ravdess에 대한 데이터 경로 지정
data_path.to_csv("data_path.csv",index=False)

데이터 변형

In [ ]:
def noise(data): # 노이즈 추가
  noise_amp=0.35*np.random.uniform()*np.amax(data)
  data=data+noise_amp*np.random.normal(size=data.shape[0])
  return data

def stretch(data,rate=0.8): #샘플의 길이 변형
  return librosa.effects.time_stretch(data,rate=rate)

def shift(data): #샘플의 시간이동
  shift_range=int(np.random.uniform(low=-5,high=5)*1000)
  return np.roll(data,shift_range)

def pitch(data,smpling_rate,pitch_factor=0.7): #목소리 메인 주파수 변형
  return librosa.effects.pitch_shift(data,sr=sampling_rate,n_steps=pitch_factor)

path=np.array(data_path.Path)[1]
data,sample_rate=librosa.load(path)

특징 추출


In [ ]:
def extract_features(data):
  result=np.array([])

  #Zero crossing rate
  zcr=np.mean(librosa.feature.zero_crossing_rate(y=data).T,axis=0)
  result=np.hstack((result,zcr))

  #단시간 푸리에 변환
  stft=np.abs(librosa.stft(data))
  chroma_stft=np.mean(librosa.feature.chroma_stft(S=stft,sr=sample_rate).T,axis=0)
  result=np.hstack((result,chroma_stft))

  #실효값
  rms=np.mean(librosa.feature.rms(y=data).T,axis=0)
  result=np.hstack((result,rms))

  #멜 필터를 적용한 스펙트럼 값
  mfcc=np.mean(librosa.feature.rms(y=data).T,axis=0)
  result=np.hstack((result,mfcc))

  #멜 스텍트럼
  mel=np.mean(librosa.feature.melspectrogram(y=data,sr=sample_rate).T,axis=0)
  result=np.hstack((result,mel))

  return result


def get_features(path):
  data,sample_rate=librosa.load(path,duration=2.5,offset=0.6)

  res1=extract_features(data)
  result=np.array(res1)

  noise_data=noise(data)
  res2=extract_features(noise_data)

  new_data=stretch(data)
  data_stretch_pitch=pitch(new_data,sample_rate)
  res3=extract_features(data_stretch_pitch)
  result=np.vstack((result,res3))

  return result

In [ ]:
filtered_row = data_path[data_path['Emotions'] == emotion]
if not filtered_row.empty:
    path = filtered_row.iloc[0]['Path']
data, sampling_rate = librosa.load(path)
X,Y=[],[]
for path,emotion in zip(data_path.Path,data_path.Emotions):
  feature=get_features(path)
  for ele in feature:
    X.append(ele)
    Y.append(emotion)

NameError: ignored

In [ ]:
len(X),len(Y),data_path.Path.shape

In [ ]:
Features=pd.DataFrame(X)
Features['labels']=Y
Features.to_csv('features.cs',index=False)

In [ ]:
X=Features.iloc[:,:-1].values
Y=Features['labels'].values

In [ ]:
encoder=OneHotEncoder()
Y=encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=0,shuffle=True)
x_train.shape, y_train.shape,x_test.shape,y_test.shape

In [ ]:
scalar=StandardScaler()
x_train=scalar.fit_transform(x_train)
x_test=scalar.transform(x_test)
x_train.shape, y_train.shape,x_test.shape,y_test.shape

In [ ]:
x_train=np.expand_dims(x_train,axis=2)
x_test=np.expand_dims(x_test,axis=2)
x_train.shape, y_train.shape,x_test.shape,y_test.shape

기계학습 모델링

In [ ]:
model=Sequential()

model.add(Conv1D(512,kernel_size=5, strides=1,padding='same', activation='relu',input_shape=(x_train.shape[1],1)))
model.add(MaxPooling1D(pool_size=5,strides=2,padding='same'))

model.add(Conv1D(512,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=5,strides=2,padding='same'))
model.add(Dropout(0.2))

model.add(Conv1D(256,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=5,strides=2,padding='same'))
model.add(Dropout(0.2))

model.add(Conv1D(128,kernel_size=3,strides=1,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=3,strides=2,padding='same'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=8, activation='softmax'))

model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model.summary()

In [ ]:
rlrp=ReduceLROnPlateau(
    monitor='loss',
    factor=0.4,
    verbose=0,
    patience=2,
    min_lr=0.0000001)
history=model.fit(x_train,y_train,batch_size=500,
                  epochs=30,
                  validation_data=(x_test,y_test),
                  callbacks=[rlrp])

In [ ]:
print("Accuracy of our model on test data : " , model.evaluate(x_test,y_test)[1]*100 , "%")

epochs = [i for i in range(30)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
test_acc = history.history['val_accuracy']
test_loss = history.history['val_loss']

fig.set_size_inches(20,6)
ax[0].plot(epochs , train_loss , label = 'Training Loss')
ax[0].plot(epochs , test_loss , label = 'Testing Loss')
ax[0].set_title('Training & Testing Loss')
ax[0].legend()
ax[0].set_xlabel("Epochs")

ax[1].plot(epochs , train_acc , label = 'Training Accuracy')
ax[1].plot(epochs , test_acc , label = 'Testing Accuracy')
ax[1].set_title('Training & Testing Accuracy')
ax[1].legend()
ax[1].set_xlabel("Epochs")
plt.show()

In [ ]:
pred_test = model.predict(x_test)
y_pred = encoder.inverse_transform(pred_test)

y_test = encoder.inverse_transform(y_test)

In [ ]:
df = pd.DataFrame(columns=['Predicted Labels', 'Actual Labels'])
df['Predicted Labels'] = y_pred.flatten()
df['Actual Labels'] = y_test.flatten()

df.head(10)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize = (12, 10))
cm = pd.DataFrame(cm , index = [i for i in encoder.categories_] , columns = [i for i in encoder.categories_])
sns.heatmap(cm, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')
plt.title('Confusion Matrix', size=20)
plt.xlabel('Predicted Labels', size=14)
plt.ylabel('Actual Labels', size=14)
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))